<a href="https://colab.research.google.com/github/silverstar0727/pragmatism-AI/blob/main/ResNet(cifar100).ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

In [1]:
!pip install -q wandb

     |████████████████████████████████| 1.7 MB 4.2 MB/s 
     |████████████████████████████████| 139 kB 69.1 MB/s 
     |████████████████████████████████| 97 kB 6.7 MB/s 
     |████████████████████████████████| 180 kB 72.6 MB/s 
     |████████████████████████████████| 63 kB 1.8 MB/s 


In [2]:
import tensorflow as tf
from sklearn.model_selection import train_test_split
from tensorflow import keras

from tensorflow.keras.layers import Input, Conv2D, ReLU, BatchNormalization,\
                                    Add, AveragePooling2D, Flatten, Dense
from tensorflow.keras.models import Model

from wandb.keras import WandbCallback
import wandb

In [3]:
cifar100 = tf.keras.datasets.cifar100

(x_train, y_train), (x_test, y_test) = cifar100.load_data()

x_train, x_test = x_train / 255.0, x_test / 255.0
x_train, x_val, y_train, y_val = train_test_split(x_train, y_train, test_size=0.2, random_state=3)


# Prepare dataloader
trainloader = tf.data.Dataset.from_tensor_slices((x_train, y_train))
trainloader = trainloader.shuffle(1024).batch(1024).prefetch(tf.data.experimental.AUTOTUNE)
valloader = tf.data.Dataset.from_tensor_slices((x_val, y_val))
valloader = valloader.batch(1024).prefetch(tf.data.experimental.AUTOTUNE)
testloader = tf.data.Dataset.from_tensor_slices((x_test, y_test))
testloader = testloader.batch(1024).prefetch(tf.data.experimental.AUTOTUNE)

169017344/169001437 [==============================] - 11s 0us/step


In [4]:
def relu_bn(x):
    x = ReLU()(x)
    x = BatchNormalization()(x)
    return x

def residual_block(x, downsample, filters, kernel_size=3):
    y = Conv2D(kernel_size=kernel_size,
               strides= (1 if not downsample else 2),
               filters=filters,
               padding="same")(x)
    y = relu_bn(y)
    y = Conv2D(kernel_size=kernel_size,
               strides=1,
               filters=filters,
               padding="same")(y)

    if downsample:
        x = Conv2D(kernel_size=1,
                   strides=2,
                   filters=filters,
                   padding="same")(x)
    out = Add()([x, y])
    out = relu_bn(out)
    return out

def create_res_net():
    
    inputs = Input(shape=(32, 32, 3))
    num_filters = 64
    
    x = BatchNormalization()(inputs)
    x = Conv2D(kernel_size=3,
               strides=1,
               filters=num_filters,
               padding="same")(x)
    x = relu_bn(x)
    
    num_blocks_list = [2, 5, 5, 2]
    for i in range(len(num_blocks_list)):
        num_blocks = num_blocks_list[i]
        for j in range(num_blocks):
            x = residual_block(x, downsample=(j==0 and i!=0), filters=num_filters)
        num_filters *= 2
    
    x = AveragePooling2D(4)(x)
    x = Flatten()(x)
    outputs = Dense(100, activation='softmax')(x)
    
    model = Model(inputs, outputs)

    return model

In [5]:
def experiment():
    wandb.init(project='ResNet', magic=True)
    args = wandb.config

    # init
    keras.backend.clear_session()
    
    net = create_res_net()
    optimizer = tf.keras.optimizers.Adam(learning_rate=args.lr)
    net.compile(optimizer=optimizer,
                loss='sparse_categorical_crossentropy',
                metrics=['accuracy'])
    
    result = net.fit(
        trainloader, 
        epochs=args.epochs, 
        validation_data=valloader, 
        batch_size=1024,
        callbacks=[WandbCallback(save_model=True)])

    test_result = net.evaluate(testloader, callbacks=[WandbCallback(save_model=False)])

In [6]:
sweep_config = {
    'method': 'grid',
    'parameters': {
        'lr': {
            'values': [0.001, 0.0001]
        },
        'epochs': {
            'values': [20]
        }
    }
}

sweep_id = wandb.sweep(sweep_config)
wandb.agent(sweep_id, function=experiment)

<IPython.core.display.Javascript object>

wandb: Appending key for api.wandb.ai to your netrc file: /root/.netrc


Create sweep with ID: w5vxjxy4
Sweep URL: https://wandb.ai/jeongmin/uncategorized/sweeps/w5vxjxy4


wandb: Agent Starting Run: 8a8abyrh with config:
wandb: 	epochs: 20
wandb: 	lr: 0.001
wandb: Currently logged in as: jeongmin (use `wandb login --relogin` to force relogin)
wandb: WARNING Ignored wandb.init() arg project when running a sweep


wandb: WARNING Ignored wandb.init() arg project when running a sweep


...

wandb: Agent Starting Run: kudtv034 with config:
wandb: 	epochs: 20
wandb: 	lr: 0.0001
wandb: WARNING Ignored wandb.init() arg project when running a sweep


Epoch 1/20
40/40 [==============================] - 57s 1s/step - loss: 4.3268 - accuracy: 0.0538 - val_loss: 4.6445 - val_accuracy: 0.0094
Epoch 2/20
40/40 [==============================] - 53s 1s/step - loss: 3.8151 - accuracy: 0.1280 - val_loss: 4.7678 - val_accuracy: 0.0094
Epoch 3/20
40/40 [==============================] - 53s 1s/step - loss: 3.4252 - accuracy: 0.1953 - val_loss: 4.9576 - val_accuracy: 0.0110
Epoch 4/20
40/40 [==============================] - 53s 1s/step - loss: 3.0406 - accuracy: 0.2745 - val_loss: 5.2696 - val_accuracy: 0.0110
Epoch 5/20
40/40 [==============================] - 53s 1s/step - loss: 2.6531 - accuracy: 0.3656 - val_loss: 5.2923 - val_accuracy: 0.0081
Epoch 6/20
40/40 [==============================] - 53s 1s/step - loss: 2.2530 - accuracy: 0.4615 - val_loss: 5.3759 - val_accuracy: 0.0135
Epoch 7/20
40/40 [==============================] - 53s 1s/step - loss: 1.9747 - accuracy: 0.5244 - val_loss: 5.5968 - val_accuracy: 0.0243
Epoch 8/20
40/40 [==

accuracy,0.98002
best_epoch,11
best_val_loss,4.06951
epoch,19
loss,0.12578
val_accuracy,0.2043
val_loss,4.57847


accuracy,▁▂▂▃▃▄▅▅▅▅▆▆▆▇▇▇████
epoch,▁▁▂▂▂▃▃▄▄▄▅▅▅▆▆▇▇▇██
loss,█▇▆▆▅▅▄▄▃▃▃▃▂▂▂▁▁▁▁▁
val_accuracy,▁▁▁▁▁▁▂▂▃▅▅▇▇▇██████
val_loss,▄▄▅▇▇▇█▆▄▃▂▁▁▂▂▂▂▃▃▃


wandb: Sweep Agent: Waiting for job.
wandb: Sweep Agent: Exiting.
